In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import mlflow
import mlflow.tensorflow
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

In [6]:
# Data generators with InceptionV3 expected input size
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    'D://PROJECTS/ALL_PROJECTS/FREEZING_AND_NON_FREEZING/wavelet_training/data/train',
    target_size=(299, 299),  # Adjusted for InceptionV3
    batch_size=16,
    class_mode='binary'
)

validation_generator = valid_datagen.flow_from_directory(
    'D://PROJECTS/ALL_PROJECTS/FREEZING_AND_NON_FREEZING/wavelet_training/data/valid',
    target_size=(299, 299),  # Adjusted for InceptionV3
    batch_size=16,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    'D://PROJECTS/ALL_PROJECTS/FREEZING_AND_NON_FREEZING/wavelet_training/data/test',
    target_size=(299, 299),  # Adjusted for InceptionV3
    batch_size=16,
    class_mode='binary',
    shuffle=False
)


Found 3583 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [12]:
mlflow.set_experiment(experiment_id="322113877408650181")
 # set the experiment id from the mlflow website whenever new experiment is created 


<Experiment: artifact_location='mlflow-artifacts:/322113877408650181', creation_time=1708515694937, experiment_id='322113877408650181', last_update_time=1708515694937, lifecycle_stage='active', name='FREEZING_AND_NON_FREEZING_SIGNAL_IMAGES_PROJECT', tags={}>

In [11]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.tensorflow.autolog()

In [14]:
from tensorflow.keras.layers import Dropout

# Updated CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Initial Training
with mlflow.start_run(run_name="cropped_changed_TO_WAVELET_DATA_with_batch_cnn_more_complex_without_finetuning"):
    model_checkpoint = ModelCheckpoint('cropped_model_changed_cnn_more_complex.h5', monitor='val_accuracy', 
                                       mode='max', save_best_only=True, verbose=1)

    early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, verbose=1, 
                                   restore_best_weights=True)

    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    history_st = model.fit(train_generator, validation_data=validation_generator, epochs=50, callbacks=[model_checkpoint, early_stopping, tensorboard_callback])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 297, 297, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 148, 148, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 146, 146, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 73, 73, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 71, 71, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 35, 35, 128)     

2024/02/23 08:20:27 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.
2024/02/23 08:20:27 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.


Epoch 1/50
224/224 [==============================] - ETA: 0s - loss: 0.6929 - accuracy: 0.5484
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to cropped_model_changed_cnn_more_complex.h5
224/224 [==============================] - 297s 1s/step - loss: 0.6929 - accuracy: 0.5484 - val_loss: 0.7014 - val_accuracy: 0.5000
Epoch 2/50
224/224 [==============================] - ETA: 0s - loss: 0.6876 - accuracy: 0.5540
Epoch 2: val_accuracy did not improve from 0.50000
224/224 [==============================] - 273s 1s/step - loss: 0.6876 - accuracy: 0.5540 - val_loss: 0.7000 - val_accuracy: 0.5000
Epoch 3/50
224/224 [==============================] - ETA: 0s - loss: 0.6880 - accuracy: 0.5540
Epoch 3: val_accuracy did not improve from 0.50000
224/224 [==============================] - 272s 1s/step - loss: 0.6880 - accuracy: 0.5540 - val_loss: 0.6985 - val_accuracy: 0.5000
Epoch 4/50
224/224 [==============================] - ETA: 0s - loss: 0.6876 - accuracy: 0.5540
Epoch 4

2024/02/23 09:08:26 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: '>=' not supported between instances of 'slice' and 'int'
2024/02/23 09:08:26 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\ravit\AppData\Local\Temp\tmpezldfptq\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\ravit\AppData\Local\Temp\tmpezldfptq\model\data\model\assets
